In [537]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import sklearn
from sklearn.naive_bayes import GaussianNB

In [ ]:
chitchat_df = pd.read_pickle("./chitchat.pkl")

In [ ]:
chitchat_df.head()

In [ ]:
chitchat_df.drop(['prompt'], axis=1, inplace=True)

In [ ]:
chitchat_df


In [ ]:
cc_new = pd.DataFrame().assign(X=chitchat_df['query'])

In [ ]:
cc_new["Y"] = 0

In [570]:
cc_new = cc_new[["Y" ,"X"]]

In [571]:
cc_new.head()

,Y,X
0,0,Hello
1,0,How are you doing today?
2,0,whats up MD
3,0,"Im alright, I just took a nap. But it was one ..."
4,0,oh wow haha


In [ ]:
reddit_df = pd.read_pickle("./final.pkl")

In [ ]:
reddit_df.head()

In [ ]:
reddit_df.shape[0] - reddit_df['selftext'].isna().sum()

In [ ]:
reddit_df = reddit_df.loc[reddit_df['is_submission'] == False][["parent_body","body"]]

In [ ]:
reddit_df.head()

In [ ]:
reddit_df.rename(columns={"parent_body": "query", "body": "response"}, inplace=True)

In [ ]:
merged_qr_df = pd.concat([chitchat_df, reddit_df])

In [516]:
merged_qr_df

,query,response
0,Hello,whats up MD
1,How are you doing today?,im doing good
2,whats up MD,"Im alright, I just took a nap. But it was one ..."
3,"Im alright, I just took a nap. But it was one ...",oh wow haha
4,oh wow haha,Yeah
...,...,...
1412,,"Check out her early stuff. Less pop, more int..."
1413,I fucking love Shakira. She is a national tre...,Éres de Colombia? También soy un fan de ella! ...
1414,,I fucking love Shakira. She is a national tre...
1416,,"Well, the hips don't lie."


In [ ]:
reddit_new = pd.DataFrame().assign(X=reddit_df['parent_body'])

In [ ]:
reddit_new = pd.DataFrame().assign(X=reddit_df['query'])

In [ ]:
reddit_new["Y"] = 1

In [ ]:
reddit_new = reddit_new[["Y", "X"]]

In [ ]:
reddit_new

In [ ]:
reddit_new.tail(30)

In [ ]:
reddit_df.loc[reddit_df['is_submission'] == False]["body"]

In [ ]:
reddit_new = pd.DataFrame().assign(X=reddit_df['selftext'])


In [ ]:
# reddit_df['selftext'].isna().sum()
val = reddit_df['body'].isna().sum()
val

In [561]:
res_df = pd.concat([reddit_new, cc_new])
res_df.drop_duplicates(inplace=True)

In [ ]:
import collections
from nltk.stem import PorterStemmer
import re
import contractions

In [562]:
def remove_emojis(data):
        emoj = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002500-\U00002BEF"  # chinese char
            u"\U00002702-\U000027B0"
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U0001f926-\U0001f937"
            u"\U00010000-\U0010ffff"
            u"\u2640-\u2642" 
            u"\u2600-\u2B55"
            u"\u200d"
            u"\u23cf"
            u"\u23e9"
            u"\u231a"
            u"\ufe0f"  # dingbats
            u"\u3030"
                        "]+", re.UNICODE)
        return re.sub(emoj, '', data)

def cleaner(text):    
    text = contractions.fix(text)
    text = text.replace("&gt;", "")
    text = text.replace("&lt;", "")
    text = text.replace("\n", " ")

    regex_url = r"(http[^\s]+)"
    wo_url = re.sub(regex_url,"",text)
    wo_url = re.sub(r"#","",wo_url)
    # wo_url = re.sub(r"(@[^\s]+)","",wo_url)
    wo_url = re.sub(r"(@[^\s]+)","@USER_",wo_url)

    return wo_url
    # all_emoticons = [':/',':l',':-)','-.-',':::---))))','&lt;',":'(",'-_-',':)','xx','xxx',';-)',':(',':-/',':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    #         ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D','d:',':-l','&lt;3',
    #         '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',':|','o_o',';)',
    #         'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',':-0','-__-',':(((',
    #         '<3'':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    #         ':-[', ':-<', '=/', '>:(', ':(', '>.<', ":'-(", ":'(",  ':-c',
    #         ':c', ':{',  ';(', "=\\", ">:\\", ':\\']
        
    # a = wo_url
    # for i in all_emoticons:
    #     a = a.replace(i,'')
    #     a = remove_emojis(a)
    #     a = " ".join(a.split())

    # return a  


res_df['X'] = res_df['X'].apply(cleaner)

In [563]:
res_df

,Y,X
0,1,That is what I thought. I am sorry things have...
1,1,Nah i really do not care enough about this.
2,1,Let us hear your counter-argument then.
3,1,Yeah i do not support the premise anyways: i d...
4,1,"Your premise is invalid, which invalidates you..."
...,...,...
166163,0,12 miles is solid.
166164,0,"Oh man that is too bad, I will take lots of pi..."
166165,0,"Hey as long as you have water and time, the hi..."
166166,0,"That is so true, I hear you can pay like three..."


In [522]:
res_df.to_csv(r'A:\MS2021\SUNY BUFFALO\Third sem\Info Retrieval\Project 4\ir_project_4\backend\classifier\data.csv', header=None, index=None, sep=' ', mode='a')

In [564]:
from transformers import BertTokenizer
# using the low level BERT for our task.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [ ]:
len(stopwords.words('english'))

In [565]:
res_df

,Y,X
0,1,That is what I thought. I am sorry things have...
1,1,Nah i really do not care enough about this.
2,1,Let us hear your counter-argument then.
3,1,Yeah i do not support the premise anyways: i d...
4,1,"Your premise is invalid, which invalidates you..."
...,...,...
166163,0,12 miles is solid.
166164,0,"Oh man that is too bad, I will take lots of pi..."
166165,0,"Hey as long as you have water and time, the hi..."
166166,0,"That is so true, I hear you can pay like three..."


Data Preprocessing done, now starting with classification
------------------------------------------------------


In [566]:
#Import feature engineering modules and test_train_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

#Import classification algorithm
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
# from xgboost import XGBClassifier

#Import modules to calculate accuracy and confusion matrix
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report


NAIVE BAYES CLASSIFICATION
-----------------------------


In [567]:
tv = TfidfVectorizer(ngram_range=(1,3),max_features=20000,stop_words='english') 
X = tv.fit_transform(res_df['X'])

In [568]:
y = res_df["Y"].values
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2, shuffle=True)

In [572]:
nb = MultinomialNB(alpha=6.5, fit_prior=False)
nb.fit(Xtrain,ytrain)

MultinomialNB(alpha=6.5, fit_prior=False)

In [573]:
pred = nb.predict(Xtest)

print(accuracy_score(ytest,pred))

0.8579028189097291


In [ ]:
# changes